In [77]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MaxAbsScaler

# Preprocessing the dataset

This part of the code is essentially preparing the data for a machine learning model, transforming the text data into numerical form, and splitting the data into training and testing sets.

The combined dataset is loaded from a CSV file using pandas' read_csv function and all news headlines for each record (each day) are concatenated into a single string. A CountVectorizer is initialized to convert the headlines into a matrix of token counts. The maximum number of features is set to 5000, but this can be adjusted based on computational capacity. A LabelEncoder is used to prepare the output matrix (Y) by transforming the labels into normalized encoding.

The dataset is split into training and testing sets based on specific date ranges. The variables X_train, X_test, Y_train, and Y_test are defined in a later cell, which split the X and Y matrices into training and testing sets based on the indices of the original train and test dataframes.

In [78]:
# Load the combined dataset
data = np.squeeze(data)
data = pd.read_csv('Datasets/Combined_News_DJIA.csv')

# Concatenate all the news headlines into a single string for each record
# Optimized by directly using pandas functionality
data['All_Headlines'] = data.iloc[:, 2:].fillna('').apply(lambda x: ' '.join(x), axis=1)

# Initialize a CountVectorizer to convert the headlines to a matrix of token counts
# Consider limiting max_features and experimenting with ngram_range for better performance
vectorizer = CountVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(data['All_Headlines'])

# Prepare the output matrix
Y = LabelEncoder().fit_transform(data['Label'])

# Verify the shapes of the matrices and the first few rows to ensure the preprocessing is as expected.
print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape)
print(data[['Date', 'All_Headlines', 'Label']].head())

Shape of X: (1989, 5000)
Shape of Y: (1989,)
         Date                                      All_Headlines  Label
0  2008-08-08  b"Georgia 'downs two Russian warplanes' as cou...      0
1  2008-08-11  b'Why wont America and Nato help us? If they w...      1
2  2008-08-12  b'Remember that adorable 9-year-old who sang a...      0
3  2008-08-13  b' U.S. refuses Israel weapons to attack Iran:...      0
4  2008-08-14  b'All the experts admit that we should legalis...      1


In [79]:
# Splitting the dataset into training and testing sets
# Typically, you might want to use 80% of the data for training and 20% for testing, but these proportions can be adjusted.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardizing the features: since X is a sparse matrix returned by CountVectorizer, 
# we use MaxAbsScaler which is more appropriate for sparse data. StandardScaler is generally not used for sparse data
# because it can break the sparsity structure.

scaler = MaxAbsScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verify the standardization and splitting by printing shapes
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train_scaled shape: (1591, 5000)
X_test_scaled shape: (398, 5000)
Y_train shape: (1591,)
Y_test shape: (398,)


# Possible Baseline Model
Below is an example extension adding a simple logistic regression model as a baseline.

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Initialize the Logistic Regression model
lr_model = LogisticRegression(max_iter=1000)

# Fit the model on the scaled training data
lr_model.fit(X_train_scaled, Y_train)

# Predict on the scaled testing data
Y_pred = lr_model.predict(X_test_scaled)

# Evaluation
print("Accuracy on Test Set:", accuracy_score(Y_test, Y_pred))
print("\nClassification Report:\n", classification_report(Y_test, Y_pred))

Accuracy on Test Set: 0.46733668341708545

Classification Report:
               precision    recall  f1-score   support

           0       0.39      0.43      0.41       171
           1       0.54      0.50      0.52       227

    accuracy                           0.47       398
   macro avg       0.46      0.46      0.46       398
weighted avg       0.47      0.47      0.47       398



# Chat-GPT Based Model

We will use a Chat-GPT Based Model for predicting stock market trends from news headlines. We will use PyTorch and the Hugging Face Transformers library. Given the project's nature, we'll focus on using a pre-trained GPT model and fine-tuning it to the dataset. 

## Load and Preprocess the Dataset

We need to preprocess it into a format suitable for a GPT model. This involves tokenization and encoding the headlines, as well as preparing the labels.

Hubery run this:
pip install torch torchvision transformers


In [81]:
# Load and preprocess the data
# We need to preprocess it into a format suitable for a GPT model. 
# This involves tokenization and encoding the headlines, as well as preparing the labels.

from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Tokenizing dataset
class NewsDataset(Dataset):
    def __init__(self, headlines, labels, tokenizer, max_len):
        self.headlines = headlines
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.headlines)
    
    def __getitem__(self, item):
        headline = str(self.headlines[item])
        label = self.labels[item]
        
        encoding = self.tokenizer.encode_plus(
          headline,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
        )
        
        return {
          'headline_text': headline,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(label, dtype=torch.long)
        }


## Create Data Loaders

Preparing the dataset for training and validation. This step involves creating DataLoader instances for both training and validation sets.

In [82]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = NewsDataset(
        headlines=df.All_Headlines.to_numpy(),
        labels=df.Label.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    
    return DataLoader(ds, batch_size=batch_size, num_workers=4)

BATCH_SIZE = 16
MAX_LEN = 128

# Assuming `data` DataFrame contains the 'All_Headlines' and 'Label' columns

# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)


In [83]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load GPT-2 For Sequence Classification
# You might want to specify the number of labels depending on your task
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

# Example text
text = "This new AI model is fantastic!"

# Encode the text into tokens
inputs = tokenizer(text, return_tensors="pt")

# Predict with the model
with torch.no_grad():
    logits = model(**inputs).logits

# Convert logits to probabilities (optional)
probabilities = torch.softmax(logits, dim=-1)

# Print the probabilities
print(probabilities)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[0.0217, 0.9783]])


## Model Training

Now we will define the model training and evaluation loop. This involves loading the GPT-2 model for sequence classification, defining the optimizer, and iterating over the dataset to train the model.

In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPT2ForSequenceClassification.from_pretrained('gpt2')
model = model.to(device)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Training loop
from collections import defaultdict

def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += torch.sum(predictions == labels)
        losses.append(loss.item())
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return correct_predictions.double() / n_examples, np.mean(losses)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
